In [1]:
import pandas as pd
import networkx as nx
import itertools

In [2]:
df = pd.read_json("rawfilteredbydate.json")

In [3]:
df.head()

,caption,date
0,The Perlman Music Program Class of 2014,2014-08-22
1,Toby Perlman addresses the crowd,2014-08-22
2,Itzhak Perlman and Catherine Cho perform with...,2014-08-22
3,"Julian Beck, Lyn Barlow, Ted Ayala, Elinor He...",2014-08-22
4,Toby Perlman and guests,2014-08-22


In [4]:
df["caption"] = df['caption'].apply(lambda x: x.replace("\n",""))

In [5]:
df = df[df['caption'].apply(lambda x: len(x)<=250)]

In [193]:
restr = ["The", "Dr", "Verdana", "Helvetica" ,"Chairman","Director",
         "Guests","President","Hospital","Special Surgery","Executive Director"
         ,"New York","Museum","Photographs","Children","The Society","Dinner",
        "Board Member","Castle","President and CEO","Directors","Clockwise","Doubles","BFAnyc","Standing",
        "Susan Magazine","Trustees","Chicago","Yugoslavia","Steering Committee",
        "I Have A Dream"]

In [194]:
prereplace = ["friend","friends"]

In [195]:
def replaceWord(s):
    for i in range(len(prereplace)):
        s = s.replace(prereplace[i],"")
    return s

In [196]:
captions = map(lambda x:replaceWord(x),df.caption)

In [197]:
proper_noun_regex = r'([A-Z][A-Za-z-]*(,?\s+(\b(and|von|de|van der|van|der)\b))?(\s[A-Z][A-Za-z\'-]*)*)'

In [198]:
import re

In [199]:
G=nx.MultiGraph()
for caption in captions:
    name = []
    caption_temp  = re.findall(proper_noun_regex,caption)
    for j in range(len(caption_temp)):
        name_temp = caption_temp[j]
        if len(name_temp[0])>5 and len(name_temp[0].split(" ")) > 1 and len(name_temp[0].split(" ")) <=4:
            
            if name_temp[0] == "PatrickMcMullan":
                name.append("Patrick McMullan")
            elif name_temp[0] == "AnnieWatt":
                name.append("Annie Watt")
            elif "Mayor" in name_temp[0]:
                temp = name_temp[0].replace("Mayor ","")
                name.append(temp)
            elif len(name_temp[0].split(" and ")) == 2:
                name1 = name_temp[0].split(" and ")[0]
                name2 = name_temp[0].split(" and ")[1]
                if len(name1.split(" ")) == 1 and len(name2.split(" ")) == 2:
                    name.append(name1+ " " + name2.split(" ")[1])
                    name.append(name2)
            else:
                name.append(name_temp[0])
        for word in restr:
            if word in name:
                name.remove(word)
    if len(name) > 2:
        _=[G.add_edge(c[0],c[1],weight=1) for c in itertools.combinations(name,2)]
    elif len(name) > 1:
        G.add_edge(name[0],name[1],weight=1)        

In [200]:
Gg = nx.Graph()
for u,v,data in G.edges_iter(data=True):
    if Gg.has_edge(u,v):
        Gg[u][v]['weight'] += 1
    else:
        Gg.add_edge(u, v, weight=1)

## degrees

In [201]:
degrees=pd.DataFrame(G.degree().values(),index=G.degree().keys()).sort(0,ascending=False)

In [202]:
degrees = degrees.reset_index()

In [203]:
degrees[180:220]

,index,0
180,Linda Fargo,107
181,Karen Pearl,107
182,Kelly Rutherford,107
183,Amy Sacco,106
184,Ann Rapp,106
185,Veronica Kelly,106
186,Christine Schwarzman,105
187,Kirk Henckels,105
188,Heather Mnuchin,105
189,Donna Hanover,104


In [204]:
degrees.columns = ["name","number"]

In [205]:
[i for i in degrees.name if " and " in i]

[u'Vice-Governor and Deputy Mayor']

In [206]:
degreeres = map(lambda x,y: (x,y),degrees.name[0:100], degrees.number[0:100])

In [207]:
import pickle

In [208]:
with open('degreeres.pickle', 'wb') as f:
    pickle.dump(degreeres, f)

In [209]:
pr=nx.pagerank(Gg)
pagerank=pd.DataFrame(pr.values(),index=pr.keys()).sort(0,ascending=False)

In [210]:
pagerank.head()

,0
Jean Shafiroff,0.000642
Patrick McMullan,0.000533
Mark Gilbertson,0.000505
Gillian Miniter,0.000447
Geoffrey Bradfield,0.000367
